In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ======================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [2]:
import os
import glob

import numpy as np
import pandas as pd

import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = os.environ.get("NUM_ROWS", 100000)

In [5]:
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., int(NUM_ROWS)).astype(np.int32), 1, 50000)
# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, int(NUM_ROWS)), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['age_days'] = np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)
df['weekday_sin']= np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [6]:
df.head()

,session_id,item_id,category,age_days,weekday_sin,day
0,88469,7,2,0.876678,0.330926,3
1,83904,120,21,0.258138,0.422327,6
2,81493,42,8,0.234976,0.686702,6
3,73668,182,32,0.498537,0.121023,8
4,74926,59,11,0.126336,0.765945,1


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers. Note that the `Categorify` op encodes OOVs or nulls to `0` automatically. In our synthetic dataset we do not have any nulls. On the other hand `0` is also used for padding the sequences in input block, therefore, you can set `start_index=1` arg in the Categorify op if you want the encoded null or OOV values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

Here our goal is to create sequential features.  In this cell, we are creating temporal features and grouping them together at the session level. In this synthetically generated example dataset, we do not have a timestamp column, but if we had it (that's the case for most of the real-world datasets), we would be sorting the interactions by the timestamp column as in this [example notebook](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/end-to-end-session-based/01-ETL-with-NVTabular.ipynb). Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [7]:
SESSIONS_MAX_LENGTH =20

# Categorify categorical features
categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify()

# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'age_days', 'weekday_sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "age_days": ["list"],
        'weekday_sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['category-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
)

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['age_days-list', 'weekday_sin-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['item_id-list', 'category-list', 'age_days-list', 'weekday_sin-list'] >>  nvt.ops.ValueCount()


workflow = nvt.Workflow(filtered_sessions['session_id', 'day-first', 'item_id-count'] + seq_feats_list)

dataset = nvt.Dataset(df, cpu=False)
# Generate statistics for the features
workflow.fit(dataset)
# Apply the preprocessing and return an NVTabular dataset
sessions_ds = workflow.transform(dataset)
# Convert the NVTabular dataset to a Dask cuDF dataframe (`to_ddf()`) and then to cuDF dataframe (`.compute()`)
sessions_gdf = sessions_ds.to_ddf().compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [8]:
sessions_gdf.head(3)

,session_id,day-first,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,1,1,19,"[27, 26, 7, 46, 13, 2, 4, 237, 10, 35, 46, 35,...","[5, 5, 2, 8, 2, 1, 2, 45, 3, 6, 8, 6, 1, 16, 3...","[0.97853184, 0.4591664, 0.083990775, 0.7000025...","[0.04896013, 0.18139902, 0.5046173, 0.48253214..."
1,2,4,18,"[45, 7, 17, 17, 44, 17, 35, 23, 1, 194, 13, 18...","[8, 2, 4, 4, 8, 4, 6, 4, 2, 36, 2, 4, 2, 7, 7,...","[0.4287106, 0.75681955, 0.70978284, 0.8613602,...","[0.88505965, 0.07815777, 0.8684893, 0.25832322..."
2,3,5,15,"[27, 38, 10, 64, 84, 109, 152, 37, 13, 158, 3,...","[5, 7, 3, 11, 15, 19, 29, 7, 2, 27, 1, 4, 5, 2...","[0.07284722, 0.48706728, 0.09015047, 0.4159043...","[0.4339925, 0.51614755, 0.3565242, 0.5322814, ..."


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [9]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(Tags.CATEGORICAL),"DType(name='int64', element_type=<ElementType....",False,False,NaN,0.0,0.0,0.0,.//categories/unique.session_id.parquet,0.0,19871.0,session_id,19872.0,408.0,NaN,NaN
1,day-first,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-count,(Tags.CATEGORICAL),"DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,502.0,item_id,503.0,52.0,NaN,NaN
3,item_id-list,"(Tags.CATEGORICAL, Tags.ITEM_ID, Tags.ID, Tags...","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,502.0,item_id,503.0,52.0,2.0,19.0
4,category-list,"(Tags.LIST, Tags.CATEGORICAL)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,0.0,.//categories/unique.category.parquet,0.0,125.0,category,126.0,24.0,2.0,19.0
5,age_days-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,19.0
6,weekday_sin-list,"(Tags.LIST, Tags.CONTINUOUS)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,19.0


The following will generate `schema.pbtxt` file in the provided folder.

In [10]:
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [11]:
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow_etl"))

## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [12]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))

In [13]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 27.95it/s]


## Checking the preprocessed outputs

In [14]:
TRAIN_PATHS = os.path.join(OUTPUT_DIR, "1", "train.parquet")

In [15]:
df = pd.read_parquet(TRAIN_PATHS)
df

,session_id,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,1,19,"[27, 26, 7, 46, 13, 2, 4, 237, 10, 35, 46, 35,...","[5, 5, 2, 8, 2, 1, 2, 45, 3, 6, 8, 6, 1, 16, 3...","[0.97853184, 0.4591664, 0.083990775, 0.7000025...","[0.04896013, 0.18139902, 0.5046173, 0.48253214..."
1,17,13,"[15, 5, 5, 58, 8, 18, 29, 34, 2, 3, 43, 54, 9]","[3, 1, 1, 10, 1, 4, 6, 6, 1, 1, 8, 10, 1]","[0.76496226, 0.85960853, 0.13536207, 0.3988903...","[0.3081522, 0.17396946, 0.8448347, 0.8297997, ..."
2,34,13,"[17, 12, 9, 21, 29, 6, 23, 6, 5, 176, 12, 26, 1]","[4, 3, 1, 4, 6, 1, 4, 1, 1, 29, 3, 5, 2]","[0.42529476, 0.66954064, 0.46188155, 0.2200255...","[0.951742, 0.7311401, 0.6795269, 0.5283087, 0...."
4,58,12,"[84, 11, 7, 66, 23, 1, 36, 5, 19, 22, 6, 22]","[15, 3, 2, 12, 4, 2, 7, 1, 4, 4, 1, 4]","[0.7655469, 0.4924979, 0.9192873, 0.6521773, 0...","[0.060284566, 0.9057582, 0.9853312, 0.27452144..."
5,64,12,"[12, 7, 6, 5, 26, 20, 90, 28, 132, 36, 21, 8]","[3, 2, 1, 1, 5, 2, 16, 5, 23, 7, 4, 1]","[0.86268437, 0.11732827, 0.31621945, 0.0408642...","[0.8027563, 0.7638514, 0.055432655, 0.06549974..."
...,...,...,...,...,...,...
2110,19074,2,"[10, 16]","[3, 3]","[0.9952336, 0.018463716]","[0.3855745, 0.8623388]"
2111,19122,2,"[37, 28]","[7, 5]","[0.26565734, 0.3376144]","[0.519952, 0.117240556]"
2112,19128,2,"[18, 15]","[4, 3]","[0.65739745, 0.46439078]","[0.49096248, 0.5064814]"
2113,19134,2,"[9, 116]","[1, 20]","[0.45008472, 0.36275008]","[0.10166882, 0.8127918]"


In [16]:
import gc
del df
gc.collect()

490

You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model. Please shut down this kernel to free the GPU memory before you start the next one.